# Example: VQE program

This tutorial will be demonstation of creating VQE Program as well as migration guide on how you can replicate IBM Quantum VQE custom runtime program.

Let's first get information on what is VQE runtime program and what inputs and outputs it has.

**Description** of runtime program is: Variational Quantum Eigensolver (VQE) to find the minimal eigenvalue of a Hamiltonian.

**Inputs**:

| name | type | description |
| ---- | ---- | ----------- |
| ansatz | object | A parameterized quantum circuit preparing the ansatz wavefunction for the VQE. It is assumed that all qubits are initially in the 0 state. | 
| initial_parameters|[array,string]|Initial parameters of the ansatz. Can be an array or the string ``'random'`` to choose random initial parameters.|
|operator|object|The Hamiltonian whose smallest eigenvalue we're trying to find. Should be PauliSumOp|
|method|str|The classical optimizer used in to update the parameters in each iteration. |


**Return values**

| name | type | description |
| ---- | ---- | ----------- |
|cost_function_evals|integer|The number of cost function (energy) evaluations.|
|optimal_parameters|null|Not supported at the moment, therefore ``None``.|
|optimal_point|array|The optimal parameter values found during the optimization. This is a numpy array.|
|optimal_value|number|The smallest value found during the optimization. Equal to the ``eigenvalue`` attribute. This is a float.|
|optimizer_evals|integer|The number of steps of the optimizer.|
|optimizer_history|object|A dictionary containing information about the function evaluations (not necessarily the actual parameter value!): the current evaluation count, the parameters, the energy and the standard deviation.|
|optimizer_time|number|The total time taken by the optimizer. This is a float.|

We will also add optional `QiskitRuntimeService` as an argument to use that to access real devices.


With that information we can start drafting our program implementation in `vqe.py` file.

What our program should do:
1. parse input arguments
2. create run_vqe function that accepts estimator instance, creates VQE and runs calculation
3. decide which estimator to use and run vqe
    - if runtime service was passed then create a session and run `run_vqe` function
    - if runtime service was not passed then use stantard qiskit estimator
4. save results from vqe

Roughly our VQE program will look like this

```python
# vqe.py

import ...

def run_vqe(
    initial_parameters,
    ansatz,
    operator,
    estimator,
    method
):
    ...

arguments = get_arguments()

service = arguments.get("service")
ansatz = arguments.get("ansatz")
operator = arguments.get("operator")
initial_parameters = arguments.get("initial_parameters") 
optimizer = ...

...

if service is not None:
    # if we have service we need to open a session and create estimator
    backend = arguments.get("backend", "ibmq_qasm_simulator")
    with Session(service=service, backend=backend) as session:
        estimator = Estimator(session=session, options=options) # qiskit_ibm_runtime.Estimator
        vqe_result = run_vqe( estimator=estimator, ...)
else:
    # if we do not have a service let's use standart local estimator
    estimator = QiskitEstimator() # qiskit.primitives.Estimator

vqe_result, callback_dict = run_vqe(
    initial_parameters=initial_parameters,
    ansatz=ansatz,
    operator=operator,
    estimator=estimator,
    method=method
)

save_result({
    "optimal_point": vqe_result.x.tolist(),
    "optimal_value": vqe_result.fun,
    "optimizer_evals": vqe_result.nfev,
    "optimizer_history": callback_dict.get("cost_history", []),
    "optimizer_time": callback_dict.get("_total_time", 0)
})


```

At this point we have our program implemented. Now we need to actually run it. But before let's prepare input arguments from our VQE program.

In [1]:
import numpy as np


from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Session, Options

USE_RUNTIME_SERVICE = False

service = None
if USE_RUNTIME_SERVICE:
    service = QiskitRuntimeService()
    backend = "ibmq_qasm_simulator"

operator = SparsePauliOp.from_list(
    [("YZ", 0.3980), ("ZI", -0.3980), ("ZZ", -0.0113), ("XX", 0.1810)]
)
ansatz = EfficientSU2(operator.num_qubits)
    
input_arguments = {
    "ansatz": ansatz,
    "operator": operator,
    "method": "COBYLA",
    "service": service
}

input_arguments

{'ansatz': <qiskit.circuit.library.n_local.efficient_su2.EfficientSU2 at 0x7f7e48372ee0>,
 'operator': SparsePauliOp(['YZ', 'ZI', 'ZZ', 'XX'],
               coeffs=[ 0.398 +0.j, -0.398 +0.j, -0.0113+0.j,  0.181 +0.j]),
 'method': 'COBYLA',
 'service': None}

With arguments prepared we can create our quantum serverless client, setup provider and run our program

In [2]:
from quantum_serverless import QuantumServerless, Provider
import os

In [5]:
provider = Provider(
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

In [6]:
from quantum_serverless import Program

program = Program(
    title="VQE",
    entrypoint="vqe.py",
    working_dir="./source_files/vqe/"
)

job = serverless.run(program, arguments=input_arguments)
job

<Job | 7d9da2a7-1030-410c-98b7-947ab19633b5>

In [7]:
job.status()

'QUEUED'

In [8]:
job.result()

'{"optimal_point": [5.002008197272325, 2.721826159155891, 0.0027512743626830773, 5.8913723711130705, 5.606678579501556, 1.9664899954694406, 1.722673023851201, 0.6026832255981536, 6.684414283395059, 2.9041514004005107, 5.5820245521959055, 4.054054018767433, 1.0752386408446317, 1.4443019804431008, 2.6362288712894073, 2.4038008164184803], "optimal_value": -0.7029303936682555, "optimizer_evals": 293, "optimizer_history": [0.05312275558435113, -0.37957001252970424, -0.30876990812530514, -0.37286759228704947, -0.21934473178610545, -0.28503341569019996, -0.3195688524409221, -0.2991890075969952, -0.48866933133987067, -0.33100840316233676, -0.43344011823771, -0.2534466956072254, -0.43120052460851876, -0.46996536308121023, -0.5541853766069356, -0.4674779175943805, -0.5281785300150701, -0.5422044310149498, -0.5272950454889247, -0.49267348208063916, -0.5506504305317976, -0.4599535609172526, -0.5572690659010443, -0.5048663867922246, -0.5461417224593821, -0.47633300728466815, -0.5611421990347855, -0